## 크롤링 물품 리스트 자동화 프로그램
### 요청 사항
- 물품 품목이 같은 것들을 묶어서 최대 200개까지 작성
- 카테고리 중 다이소 매장 물품인 품목을 우선적으로 선별
- 물품은 가격 순으로 정렬
- 상품 사진 추가
- 10만 원 이하의 물품만 선정

In [76]:
from openpyxl import Workbook
import pandas as pd
from io import BytesIO
from openpyxl.drawing.image import Image
import requests
import numpy as np

FILE_NAME = 'C:\\Users\\user\\Desktop\\크롤링 결과\\일반물품\\동물용품\\촬영 대상 물품 분류체계_v0.1_권혁진_다이소몰 크롤링 결과_동물용품_텍스트.xlsx'

wb = Workbook()
ws = wb.active
df = pd.read_excel(FILE_NAME, header=None)

def init(): 
    ws.append(['물품분류', '물품코드', '물품종', '순번', '상품번호', '카테고리', '상품명', '상품사진', '가격', '링크']) # 한 줄씩 넣기 위해서 만듬
    ws.column_dimensions['H'].width = 50 # 사진 담길 위치의 셀 행 크기 조절

def image(url, no): # 이미지 삽입 함수
    res = requests.get(url)
    image_data = BytesIO(res.content)
    img = Image(image_data)
    ws.add_image(img, anchor='H%s' %no)

def insert(start, data): # data -> index 가 담긴 리스트
    j = start
    for i in data:
        ws.row_dimensions[j].height = 200 # 셀의 높이 조정
        image(df[7][i], j)
        ws.append([df[0][i], df[1][i], df[2][i], df[3][i], df[4][i], df[5][i], df[6][i], '', df[8][i], df[9][i]])
        print(df[0][i], df[1][i], df[2][i], df[3][i], df[4][i], df[5][i], df[6][i], df[8][i], df[9][i])
        j = j + 1
    return j

# 특정 물품분류 항목 배열 가져오기
init()

result = list(np.unique(df[0]))
result.remove('물품분류')

start = 2 # 셀 높이 조정할 때 사용하는 변수

for temp in result:
    stuff = df.loc[df[0] == temp] # 물품목록에 해당하는 데이터 추출

    stuff_1 = stuff[stuff[5].str.contains('다이소')]
    stuff_2 = stuff[~stuff[5].str.contains('다이소')]

    # 정렬하는 플로우
    stuff_1 = stuff_1.sort_values(by=8, ascending=True)
    stuff_2 = stuff_2.sort_values(by=8, ascending=True)

    data = pd.concat([stuff_1, stuff_2])

    data = data.loc[data[8] < 100000]

    data = data.index.to_list()

    if (len(data) > 200):
        data = data[:200] # 200 개까지만 자르기

    start = insert(start, data) # 엑셀에 집어 넣기

wb.save('C:\\Users\\user\\Desktop\\result')